# Gimbal Lookup Table Calculator

This notebook gives tools to create a lookup table for each gimbal position. This lookup table is 2 dimensional where each axis describes a component of the command angle. The value associated with this position on the lookup table is a pair of servo commands which orient the gimbal into the command position.

This algorithm will first traverse all the possible servo angles and then output a table which represents the servo commands to gimbal position.

Currently this script is in it's primitive phase and only computes a gimbal angle for given servo commands.

© 2025 Ronan Howard. All rights reserved

In [1]:
import numpy as np
from scipy.optimize import minimize
from scipy.spatial.transform import Rotation
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import math

In [2]:
# positions
S_ca = np.array([21.546,-22.493,11.599])
S_cb = np.array([-22.493,21.546,11.599])
r_S = 6.45
r_arm = 22.836

# reference gimbal position
G_A_ref = np.array([19.555, 0, 15])
G_B_ref = np.array([0, 19.555, 15])
# theta_0 is angle required for flush position
theta_0 = 76.7339


In [3]:
def find_orientation(t_A, t_B):
    theta_A = np.radians(t_A + theta_0)
    theta_B = np.radians(t_B + theta_0)

    S_A = S_ca + r_S * np.array([np.sin(theta_A), np.cos(theta_A), 0])
    S_B = S_cb + r_S * np.array([np.cos(theta_B), np.sin(theta_B), 0])

    # opptimization function
    def error(q):
        R = Rotation.from_quat(q).as_matrix()
        G_A = R @ G_A_ref
        G_B = R @ G_B_ref
        return (np.linalg.norm(G_A-S_A)-r_arm)**2 + (np.linalg.norm(G_B-S_B)-r_arm)**2

    # quaternion constraint
    def constraint(q):
        return q[0]**2 + q[1]**2 + q[2]**2 + q[3]**2 - 1  # ||q|| = 1

    cons = {'type': 'eq', 'fun': constraint}

    # make initial guess (always starts with flush position)
    q0 = np.array([0,0,0,1]) # identity quaternion
    result = minimize(error, q0, method='SLSQP', constraints=cons)
    q_opt = result.x
    return Rotation.from_quat(q_opt).as_matrix()


# visualization tools
def visualize_orientation(R):
    v = R @ [0,0,1]
    print(v)
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.quiver(0,0,0, v[0], v[1], v[2])
    ax.set_xlim([-1, 1])
    ax.set_ylim([-1, 1])
    ax.set_zlim([0, 1])
    plt.show()

def visualize_orientation_vector(v):
    print(v)
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.quiver(0,0,0, v[0], v[1], v[2])
    ax.set_xlim([-0.5, 0.5])
    ax.set_ylim([-0.5, 0.5])
    ax.set_zlim([0, 1])
    plt.show()

# Lookup table calcuation

Generates lookup table

In [4]:
servo_range = 10

servo_values = [x for x in range(-servo_range, servo_range + 1, 1)]

# this will be a 2 dimensional array
# first dimension describes the position of the servo A and the second describes the position of servo B
# the servo values are indexed from minimum to maximum
orientations = []
inverse_table = {}

for i in range(len(servo_values)):

    orientations.append([])
    
    for j in range(len(servo_values)):
        # calculate orientation vector
        vector = find_orientation(servo_values[i], servo_values[j]) @ [0,0,1]

        # find orientation and add to array
        orientations[i].append(vector)

        # find associated gimbal command
        theta_x = math.atan2(vector[2], vector[0])
        theta_y = math.atan2(vector[2], vector[1])

        inverse_table[(servo_values[i],servo_values[j])] = (theta_x, theta_y)

lookup_table = {}

for servo_commands in inverse_table:
    lookup_table[inverse_table[servo_commands]] = servo_commands


In [5]:
# print lookup table
for key in lookup_table:
    print((float(key[0]*180/math.pi-90),float(key[1]*180/math.pi-90)), lookup_table[key])

(-9.347624081415887, -9.349756385614597) (-10, -10)
(-4.011917077995392, -11.821999486173041) (-10, -9)
(-3.6439674372931847, -11.418458613645356) (-10, -8)
(-3.3082584104259496, -10.993907291858989) (-10, -7)
(-5.973612870959116, -9.184662553864982) (-10, -6)
(-6.518765403941387, -8.047879000556819) (-10, -5)
(-6.143135425886996, -7.579793491960032) (-10, -4)
(-4.744714653906357, -7.889294144545602) (-10, -3)
(-1.5866617344473752, -8.747526598014318) (-10, -2)
(-1.2701947471196604, -8.258216984572073) (-10, -1)
(-0.4260669743499079, -7.918288441192772) (-10, 0)
(-0.6826930416484913, -7.230957963850969) (-10, 1)
(-1.2404901808836968, -6.342876981537458) (-10, 2)
(-1.616349970188864, -5.40963765580625) (-10, 3)
(-0.2834391491131356, -5.380330336010132) (-10, 4)
(1.099034584510548, -5.244917075200476) (-10, 5)
(0.9366194302946695, -4.478948937648823) (-10, 6)
(2.2473446643218296, -4.252833359456261) (-10, 7)
(0.30378623269615446, -2.4847765497810457) (-10, 8)
(1.1018104652326883, -2.1390

# Generate C++ code

This script generates instantiators for variables to be copied into a C++ file.

In [6]:
print("float lookup_keys[][2] = {")

keys = [key for key in lookup_table]

for i in range(len(keys)):
    print("\t{" + str(keys[i][0]) + ", " + str(keys[i][1]) + "}", end='')
    if i != len(keys) - 1:
        print(",")

print("\n};")
print()

print("float lookup_values[][2] = {")
for i in range(len(keys)):
    print("\t{" + str(lookup_table[keys[i]][0]) + ", " + str(lookup_table[keys[i]][1]) + "}", end='')
    if i != len(keys) - 1:
        print(",")

print("\n};")

float lookup_keys[][2] = {
	{1.4076495093354788, 1.4076122936065572},
	{1.5007751644668867, 1.364463511592099},
	{1.5071970971786703, 1.371506628484078},
	{1.5130563250261697, 1.378916446892944},
	{1.4665371139571488, 1.4104937245453215},
	{1.4570224073310212, 1.4303343404330697},
	{1.4635783872172643, 1.4385039737389556},
	{1.4879854340166072, 1.4331021683142664},
	{1.543103855413386, 1.4181231862537682},
	{1.5486272463161217, 1.426663250069996},
	{1.5633600552584803, 1.4325961223734733},
	{1.5588810854376758, 1.444592302252392},
	{1.5491456887998158, 1.4600922394181084},
	{1.5425856979505885, 1.4763803383612095},
	{1.5658493804138214, 1.4768918475865869},
	{1.5899780988878935, 1.4792552548385771},
	{1.587143419691692, 1.4926239208042225},
	{1.6100198906144392, 1.4965703821337333},
	{1.576098396777674, 1.5274287948248724},
	{1.5900265471461374, 1.5334636641329151},
	{1.5950427341576208, 1.5462608011733667},
	{1.3644644195079532, 1.5007728466115173},
	{1.4216350019593655, 1.42140153148